<a href="https://colab.research.google.com/github/sharleneanna/Image-Steganography-AES/blob/main/stg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import os
import string

img = cv2.imread("test.jpg")

msg = input("Enter secret message:")
password = input("Enter a passcode:")

d = {}
c = {}

for i in range(255):
    d[chr(i)] = i
    c[i] = chr(i)

m = 0
n = 0
z = 0

for i in range(len(msg)):
    img[n, m, z] = d[msg[i]]
    n = n + 1
    m = m + 1
    z = (z + 1) % 3

cv2.imwrite("encryptedImage.jpg", img)
os.system("start encryptedImage.jpg")

message = ""
n = 0
m = 0
z = 0

pas = input("Enter passcode for Decryption")
if password == pas:
    for i in range(len(msg)):
        message = message + c[img[n, m, z]]
        n = n + 1
        m = m + 1
        z = (z + 1) % 3
    print("Decryption message:", message)
else:
    print("YOU ARE NOT auth")

Enter secret message:hi this is sharlene here
Enter a passcode:123456
Enter passcode for Decryption123456
Decryption message: hi this is sharlene here


In [ ]:
!pip install gradio opencv-python

import cv2
import os
import gradio as gr

# Your existing steganography code
def encode_message(image, msg, password):
    img = cv2.imread(image.name)  # Read uploaded image
    d = {chr(i): i for i in range(255)}

    n, m, z = 0, 0, 0
    for i in range(len(msg)):
        img[n, m, z] = d[msg[i]]
        n += 1
        m += 1
        z = (z + 1) % 3

    encrypted_image_path = "encryptedImage.jpg"
    cv2.imwrite(encrypted_image_path, img)

    return encrypted_image_path

def decode_message(image, password_input, original_password):
    img = cv2.imread(image.name)  # Read uploaded encrypted image
    c = {i: chr(i) for i in range(255)}

    n, m, z = 0, 0, 0
    message = ""

    if password_input == original_password:
        for i in range(len(message)):
            message += c[img[n, m, z]]
            n += 1
            m += 1
            z = (z + 1) % 3
        return f"Decryption successful! Message: {message}"
    else:
        return "Incorrect password! Access denied."

# Gradio UI
encrypt_interface = gr.Interface(
    fn=encode_message,
    inputs=["file", "text", "text"],
    outputs="file",
    title="Image Steganography - Encryption",
    description="Upload an image, enter a secret message, and a password to encrypt it."
)

decrypt_interface = gr.Interface(
    fn=decode_message,
    inputs=["file", "text", "text"],
    outputs="text",
    title="Image Steganography - Decryption",
    description="Upload an encrypted image, enter the password to decrypt the hidden message."
)

gr.TabbedInterface([encrypt_interface, decrypt_interface], ["Encrypt", "Decrypt"]).launch()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 MB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.9/321.9 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 90.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 5.1 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2
Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ebbc8d9abd319b34e3.gradio.live

This share link expires in 72 hours.

In [ ]:
!pip install gradio opencv-python numpy

import cv2
import os
import gradio as gr
import numpy as np

# Function to encode the message inside an image
def encode_message(image, msg, password):
    img = cv2.imread(image.name)  # Read uploaded image
    h, w, _ = img.shape  # Get image dimensions

    # Convert message to ASCII values (list of integers)
    ascii_values = [ord(c) for c in msg]

    # Store message length in the first pixel (ensure it fits in the image)
    msg_length = len(ascii_values)
    if msg_length > h * w:
        return "Error: Message too long for this image!"

    img[0, 0, 0] = msg_length  # Store length in the first pixel

    # Embed the message starting from pixel (0,1)
    idx = 0
    for i in range(h):
        for j in range(1, w):  # Start from column 1 to leave space for length
            if idx < msg_length:
                img[i, j, 0] = ascii_values[idx]  # Store ASCII value in Red channel
                idx += 1
            else:
                break

    encrypted_image_path = "encryptedImage.jpg"
    cv2.imwrite(encrypted_image_path, img)

    return encrypted_image_path, password  # Return password for reference

# Function to decode the hidden message from an image
def decode_message(image, password_input, original_password):
    img = cv2.imread(image.name)  # Read uploaded encrypted image
    h, w, _ = img.shape  # Get image dimensions

    # Read stored message length
    msg_length = img[0, 0, 0]

    if password_input == original_password:
        message = ""
        idx = 0
        for i in range(h):
            for j in range(1, w):  # Start from column 1
                if idx < msg_length:
                    char_value = img[i, j, 0]  # Read from Red channel
                    message += chr(char_value)  # Convert back to character
                    idx += 1
                else:
                    break

        return f"Decryption successful! Message: {message}"
    else:
        return "Incorrect password! Access denied."

# Gradio UI
encrypt_interface = gr.Interface(
    fn=encode_message,
    inputs=["file", "text", "text"],
    outputs=["file", "text"],
    title="Image Steganography - Encryption",
    description="Upload an image, enter a secret message, and a password to encrypt it."
)

decrypt_interface = gr.Interface(
    fn=decode_message,
    inputs=["file", "text", "text"],
    outputs="text",
    title="Image Steganography - Decryption",
    description="Upload an encrypted image, enter the password to decrypt the hidden message."
)

gr.TabbedInterface([encrypt_interface, decrypt_interface], ["Encrypt", "Decrypt"]).launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://7591114488c871950f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
import cv2
import os
import string

img = cv2.imread("test.jpg")

msg = input("Enter secert message")

password = input("Enter password")

d={}
c={}

for i in range(255):
    d[chr(i)]=i
    c[i] = chr(i)

m=0
n=0
z=0

for i in range(len(msg)):
    img[n,m,z] = d[msg[i]]
    n=n+1
    m=m+1
    z=(z+1)%3

cv2.imwrite("Encryptedmsg.jpg",img)

os.system("start Encryptedmsg.jpg")


message =""

n=0
m=0
z=0

pas = input("Enter passcode for Decryption")

if password == pas:
    for i in range(len(msg)):
        message = message + c[img[n,m,z]]
        n=n+1
        m=m+1
        z=(z+1) % 3
    print("Decryption message",message)
else:
    print("Not valid key")

Enter secert messagehi
Enter password1234
Enter passcode for Decryption1234
Decryption message hi


In [ ]:
!pip install gradio opencv-python numpy

import cv2
import gradio as gr
import numpy as np

#Here i have encrypted the message into the image
def encode_message(image, msg, password):
    img = cv2.imread(image.name)  # read uploaded image
    h, w, _ = img.shape  # get image dimensions

    msg_ascii = [ord(c) for c in msg]  # convert message to ASCII values
    password_ascii = [ord(c) for c in password]  # convert password to ASCII values
    msg_length = len(msg_ascii)

    if msg_length + len(password_ascii) + 2 > h * w:
        return "Error: Message is too long for this image!"

    # embedding message length & password length in the first pixel
    img[0, 0, 0] = msg_length  # store message length
    img[0, 0, 1] = len(password_ascii)  # store password length

    # embedding password in the next pixels
    index = 1
    for i, char in enumerate(password_ascii):
        img[0, index, 0] = char  # store password ASCII values in the red channel
        index += 1

    # embedding the message in the remaining pixels
    index = 1 + len(password_ascii)  # start after password
    for i, char in enumerate(msg_ascii):
        img[0, index, 1] = char  # store message ASCII values in the green channel
        index += 1

    encrypted_image_path = "encrypted_image.png"
    cv2.imwrite(encrypted_image_path, img)

    return encrypted_image_path

#decrypting message from image
def decode_message(image, password_input):
    img = cv2.imread(image.name)  # read uploaded encrypted image
    h, w, _ = img.shape  # get image dimensions

    # retrieve stored message & password length
    msg_length = img[0, 0, 0]
    password_length = img[0, 0, 1]

    # retrieve stored password
    stored_password = "".join(chr(img[0, i + 1, 0]) for i in range(password_length))

    if password_input == stored_password:

        message = "".join(chr(img[0, i + 1 + password_length, 1]) for i in range(msg_length))
        return f"Decryption Successful! Message: {message}"
    else:
        return "Incorrect password! Access denied."

encrypt_interface = gr.Interface(
    fn=encode_message,
    inputs=["file", "text", "text"],
    outputs="file",
    title="Image Steganography - Encrypt",
    description="Upload an image, enter a secret message & password to hide it.",
)

decrypt_interface = gr.Interface(
    fn=decode_message,
    inputs=["file", "text"],
    outputs="text",
    title="Image Steganography - Decrypt",
    description="Upload the encrypted image & enter the correct password to reveal the message.",
)

gr.TabbedInterface([encrypt_interface, decrypt_interface], ["Encrypt", "Decrypt"]).launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://c016a92d59424fcf03.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
import cv2
import gradio as gr
import numpy as np
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
import base64
import os

def encrypt_message(secret_msg, password):
    salt = os.urandom(16)
    kdf = PBKDF2HMAC(algorithm=hashes.SHA256(), length=32, salt=salt, iterations=100000)
    key = kdf.derive(password.encode())

    iv = os.urandom(16)
    cipher = Cipher(algorithms.AES(key), modes.CBC(iv))
    encryptor = cipher.encryptor()

    padded_msg = secret_msg + ' ' * (16 - len(secret_msg) % 16)
    encrypted_msg = encryptor.update(padded_msg.encode()) + encryptor.finalize()

    return base64.b64encode(salt + iv + encrypted_msg).decode()

def decrypt_message(encrypted_msg, password):
    try:
        encrypted_msg = base64.b64decode(encrypted_msg)
        salt, iv, encrypted_data = encrypted_msg[:16], encrypted_msg[16:32], encrypted_msg[32:]

        kdf = PBKDF2HMAC(algorithm=hashes.SHA256(), length=32, salt=salt, iterations=100000)
        key = kdf.derive(password.encode())

        cipher = Cipher(algorithms.AES(key), modes.CBC(iv))
        decryptor = cipher.decryptor()
        decrypted_msg = decryptor.update(encrypted_data) + decryptor.finalize()

        return decrypted_msg.decode().strip()
    except:
        return "Decryption Failed! Incorrect Password."

def encode_message(image, msg, password):
    img = cv2.imread(image.name)
    h, w, _ = img.shape

    encrypted_msg = encrypt_message(msg, password)
    msg_ascii = [ord(c) for c in encrypted_msg]

    if len(msg_ascii) + 2 > h * w:
        return "Error: Message too long for this image!"

    img[0, 0, 0] = len(msg_ascii)

    index = 1
    for i, char in enumerate(msg_ascii):
        img[0, index, 1] = char
        index += 1

    encrypted_image_path = "encrypted_image.png"
    cv2.imwrite(encrypted_image_path, img)

    return encrypted_image_path

def decode_message(image, password_input):
    img = cv2.imread(image.name)
    h, w, _ = img.shape

    msg_length = img[0, 0, 0]
    encrypted_msg = "".join(chr(img[0, i + 1, 1]) for i in range(msg_length))

    decrypted_msg = decrypt_message(encrypted_msg, password_input)

    if decrypted_msg.startswith("Decryption Failed"):
        return decrypted_msg
    return f"Decryption Successful! Message: {decrypted_msg}"

encrypt_interface = gr.Interface(
    fn=encode_message,
    inputs=["file", "text", "text"],
    outputs="file",
    title="AES-Powered Image Steganography",
    description="Upload an image, enter a secret message & password. The message is AES-encrypted and hidden inside the image."
)

decrypt_interface = gr.Interface(
    fn=decode_message,
    inputs=["file", "text"],
    outputs="text",
    title="Decrypt AES-Encrypted Message",
    description="Upload the encrypted image & enter the correct password to retrieve your AES-encrypted message."
)

gr.TabbedInterface([encrypt_interface, decrypt_interface], ["Encrypt", "Decrypt"]).launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://0aac9e75271ec18df7.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
